## Movielens 영화 추천 실습

## 목차
1. [데이터 준비와 전처리](#데이터-준비와-전처리)
2. [분석해 봅시다.](#분석해-봅시다.)
3. [내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.](#내가-선호하는-영화를-5가지-골라서-ratings에-추가해-줍시다.)
4. [CSR matrix를 직접 만들어 봅시다.](#CSR-matrix를-직접-만들어-봅시다.)
5. [als_model \= AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.](#als_model-=-AlternatingLeastSquares-모델을-직접-구성하여-훈련시켜-봅시다.)
6. [내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.](#내가-선호하는-5가지-영화-중-하나와-그-외의-영화-하나를-골라-훈련된-모델이-예측한-나의-선호도를-파악해-보세요.)
7. [내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.](#내가-좋아하는-영화와-비슷한-영화를-추천받아-봅시다.)
8. [내가 가장 좋아할 만한 영화들을 추천받아 봅시다.](#내가-가장-좋아할-만한-영화들을-추천받아-봅시다.)
9. [회고](#회고)
## 루브릭
|평가문항|상세기준|
|:---|:---|
|1. CSR matrix가 정상적으로 만들어졌다.|사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.|
|2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.|사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.|
|3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.|MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.|

이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

## 데이터 준비와 전처리

Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [89]:
import os
import pandas as pd
rating_file_path=os.getenv('HOME') + '/aiffel/aiffel_project/exploration14/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


사용하지 않을 timestamp를 삭제

In [90]:
timestamp = ratings['timestamp']
del ratings['timestamp']

선호 기준을 3점 이상으로 설정합니다.

In [91]:
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


ratings 컬럼의 이름을 counts로 바꿉니다.

In [92]:
ratings.rename(columns={'ratings':'counts'}, inplace=True)
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

영화 제목을 보기 위해 메타 데이터를 읽어옵니다.

In [93]:
movie_file_path=os.getenv('HOME') + '/aiffel/aiffel_project/exploration14/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [94]:
ratings = pd.merge(left = ratings, right = movies, how = 'inner', on = 'movie_id')
ratings.head()

,user_id,movie_id,counts,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama


사용하지 않을 컬럼 삭제

In [95]:
del ratings['genre']
del ratings['movie_id']
ratings.head()

여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

## 분석해 봅시다.

In [99]:
# movies의 영화 수
ratings['title'].nunique()

3628

In [100]:
# 유저 수
ratings['user_id'].nunique()

6039

In [101]:
# 인기 많은 영화
movies_count = ratings.groupby('title')['user_id'].count()
group = movies_count.sort_values(ascending=False)[:30]
for i, index, value in zip(range(30), group.index, group.values):
    print(f'순위 {i+1}')
    print(f'영화 이름: {index}')
    print(f'영화를 본 사람 수: {value}')
    print('________________________________')

순위 1
영화 이름: American Beauty (1999)
영화를 본 사람 수: 3211
________________________________
순위 2
영화 이름: Star Wars: Episode IV - A New Hope (1977)
영화를 본 사람 수: 2910
________________________________
순위 3
영화 이름: Star Wars: Episode V - The Empire Strikes Back (1980)
영화를 본 사람 수: 2885
________________________________
순위 4
영화 이름: Star Wars: Episode VI - Return of the Jedi (1983)
영화를 본 사람 수: 2716
________________________________
순위 5
영화 이름: Saving Private Ryan (1998)
영화를 본 사람 수: 2561
________________________________
순위 6
영화 이름: Terminator 2: Judgment Day (1991)
영화를 본 사람 수: 2509
________________________________
순위 7
영화 이름: Silence of the Lambs, The (1991)
영화를 본 사람 수: 2498
________________________________
순위 8
영화 이름: Raiders of the Lost Ark (1981)
영화를 본 사람 수: 2473
________________________________
순위 9
영화 이름: Back to the Future (1985)
영화를 본 사람 수: 2460
________________________________
순위 10
영화 이름: Matrix, The (1999)
영화를 본 사람 수: 2434
________________________________
순위 11
영화 이름: Jurassic Park (1993)
영화를 본 

유저별 몇 개의 영화를 보았는지에 대한 통계

In [102]:
user_count = ratings.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

유저별 별점에 대한 통계

In [103]:
user_count = ratings.groupby('user_id')['counts'].median()
user_count.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: counts, dtype: float64

## 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [137]:
my_favorite = ['Star Wars: Episode VI - Return of the Jedi (1983)', 'E.T. the Extra-Terrestrial (1982)', 'Toy Story (1995)', 'Men in Black (1997)', 'Back to the Future (1985)']

선호하는 영화가 ratings 안에 있는지 확인

In [139]:
my_movies = pd.DataFrame({'user_id': [7000]*5, 'title': my_favorite, 'counts':[5]*5})
my_movies

,user_id,title,counts
0,7000,Star Wars: Episode VI - Return of the Jedi (1983),5
1,7000,E.T. the Extra-Terrestrial (1982),5
2,7000,Toy Story (1995),5
3,7000,Men in Black (1997),5
4,7000,Back to the Future (1985),5


ratings에 추가

In [140]:
if not ratings.isin({'user_id' : [6041]})['user_id'].any():
    ratings = ratings.append(my_movies)
ratings.tail(10)

,user_id,counts,title
836473,5851,5,One Little Indian (1973)
836474,5854,4,Slaughterhouse (1987)
836475,5854,3,"Promise, The (Versprechen, Das) (1994)"
836476,5938,4,"Five Wives, Three Secretaries and Me (1998)"
836477,5948,5,Identification of a Woman (Identificazione di ...
0,7000,5,Star Wars: Episode VI - Return of the Jedi (1983)
1,7000,5,E.T. the Extra-Terrestrial (1982)
2,7000,5,Toy Story (1995)
3,7000,5,Men in Black (1997)
4,7000,5,Back to the Future (1985)


유저(6039) nunique는 1 증가 tilte(3628)은 그대로인지 확인

In [142]:
ratings['title'].nunique(), ratings['user_id'].nunique()

(3628, 6040)

## CSR matrix를 직접 만들어 봅시다.

딕셔너리 생성

In [145]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

인덱싱이 잘 되었는지 확인해 봅니다.

In [148]:
print(user_to_idx[7000])
print(movie_to_idx['Star Wars: Episode VI - Return of the Jedi (1983)'])

6039
64


In [151]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')m
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

ratings

user_id column indexing Fail!!
title column indexing OK!!


,user_id,counts,title
0,0,5,0
1,1,5,0
2,2,4,0
3,3,4,0
4,4,5,0
...,...,...,...
0,6039,5,64
1,6039,5,26
2,6039,5,40
3,6039,5,175


CSR matrix 생성

In [152]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()
num_user, num_movie

(6040, 3628)

In [153]:
csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.title)), shape = (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [154]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

Implicit AlternatingLeastSquares 모델의 선언

In [161]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)

In [162]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

모델 훈련

In [163]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [164]:
user, movie_title = user_to_idx[7000], movie_to_idx['Star Wars: Episode VI - Return of the Jedi (1983)']
user_vector, movie_title_vector = als_model.user_factors[user], als_model.item_factors[movie_title]

In [165]:
user_vector, movie_title_vector

(array([ 0.40424824, -0.06490634, -0.47034132,  0.20986675,  0.8132979 ,
         0.55024993,  0.2391316 , -0.41440547, -1.2166871 , -0.9088182 ,
         0.4061006 , -0.0386149 ,  0.6257602 ,  0.08888637,  0.2062592 ,
        -0.48500752,  0.22432098,  0.29685038, -1.0243602 , -0.2573564 ,
        -0.21864308, -0.11403392, -0.21895045,  0.16315305,  0.12297557,
         0.49572986, -1.1272024 , -0.56160045, -0.576969  ,  0.18638936,
        -0.14172135,  0.10494202,  0.37574625, -0.04819255,  1.2978406 ,
         0.26225343, -0.00262833,  0.6051728 ,  0.00754029,  0.5540601 ,
        -0.14490893,  0.31617147, -0.46287826, -0.09060245, -1.012944  ,
         0.333316  , -0.66091174, -0.08032586,  0.40511543,  0.3944203 ,
         0.22427697, -0.4377735 , -0.3682739 , -0.15625249,  0.2574187 ,
         0.03523874, -0.5273937 ,  0.32959262, -0.45698297,  0.86450684,
        -0.60361123,  0.86910987,  0.3784125 ,  0.20895644, -0.08522621,
         0.8788549 ,  0.3567529 , -0.6275138 ,  0.1

In [166]:
np.dot(user_vector, movie_title_vector)

0.6961483

임의 유저가 평가하지 않은 영화 예측

In [167]:
ter = movie_to_idx['Terminator 2: Judgment Day (1991)']
ter_vector = als_model.item_factors[ter]
np.dot(user_vector, ter_vector)

0.37134656

## 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

비슷한 영화를 찾는 함수

In [188]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[favorite_movie]
    similar_movie = als_model.similar_items(movie_id, N=6)
    list_similar = [idx_to_movie[i[0]] for i in similar_movie]
    return list_similar

In [190]:
favorite_movie = 'Men in Black (1997)'
list_similar = get_similar_movie(favorite_movie)
list_similar

['Men in Black (1997)',
 'Jurassic Park (1993)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Independence Day (ID4) (1996)',
 'Fifth Element, The (1997)']

영화 장르 확인

In [191]:
def genre(similar):
    for i in similar:
        print(f"영화이름: {i}, 영화 장르: {movies[movies['title'] == i]['genre'].str.split('|')}")

In [192]:
genre(list_similar)

영화이름: Men in Black (1997), 영화 장르: 1539    [Action, Adventure, Comedy, Sci-Fi]
Name: genre, dtype: object
영화이름: Jurassic Park (1993), 영화 장르: 476    [Action, Adventure, Sci-Fi]
Name: genre, dtype: object
영화이름: Terminator 2: Judgment Day (1991), 영화 장르: 585    [Action, Sci-Fi, Thriller]
Name: genre, dtype: object
영화이름: Total Recall (1990), 영화 장르: 2847    [Action, Adventure, Sci-Fi, Thriller]
Name: genre, dtype: object
영화이름: Independence Day (ID4) (1996), 영화 장르: 770    [Action, Sci-Fi, War]
Name: genre, dtype: object
영화이름: Fifth Element, The (1997), 영화 장르: 1491    [Action, Sci-Fi]
Name: genre, dtype: object


장르 기준 비슷한 영화가 추천 되었습니다.

## 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [193]:
user = user_to_idx[7000]
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(117, 0.6039249),
 (44, 0.56601226),
 (160, 0.42454588),
 (60, 0.40876377),
 (107, 0.39040285),
 (50, 0.38402477),
 (92, 0.37134656),
 (670, 0.36249024),
 (124, 0.34532613),
 (4, 0.324086),
 (110, 0.2701946),
 (120, 0.26689392),
 (243, 0.25430694),
 (33, 0.25369847),
 (91, 0.24530245),
 (602, 0.2331081),
 (126, 0.23066689),
 (322, 0.22418502),
 (99, 0.22313745),
 (550, 0.20670758)]

In [196]:
list_title = [idx_to_movie[i[0]] for i in movie_recommended]
list_title

['Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Forrest Gump (1994)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Jurassic Park (1993)',
 'Toy Story 2 (1999)',
 'Terminator 2: Judgment Day (1991)',
 'Galaxy Quest (1999)',
 'Matrix, The (1999)',
 "Bug's Life, A (1998)",
 'Groundhog Day (1993)',
 'Raiders of the Lost Ark (1981)',
 'Ghostbusters (1984)',
 'Aladdin (1992)',
 'Close Encounters of the Third Kind (1977)',
 '2001: A Space Odyssey (1968)',
 'Shakespeare in Love (1998)',
 'Babe (1995)',
 'American Beauty (1999)',
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)']

In [197]:
genre(list_title)

영화이름: Star Wars: Episode V - The Empire Strikes Back (1980), 영화 장르: 1178    [Action, Adventure, Drama, Sci-Fi, War]
Name: genre, dtype: object
영화이름: Star Wars: Episode IV - A New Hope (1977), 영화 장르: 257    [Action, Adventure, Fantasy, Sci-Fi]
Name: genre, dtype: object
영화이름: Forrest Gump (1994), 영화 장르: 352    [Comedy, Romance, War]
Name: genre, dtype: object
영화이름: Star Wars: Episode I - The Phantom Menace (1999), 영화 장르: 2559    [Action, Adventure, Fantasy, Sci-Fi]
Name: genre, dtype: object
영화이름: Jurassic Park (1993), 영화 장르: 476    [Action, Adventure, Sci-Fi]
Name: genre, dtype: object
영화이름: Toy Story 2 (1999), 영화 장르: 3045    [Animation, Children's, Comedy]
Name: genre, dtype: object
영화이름: Terminator 2: Judgment Day (1991), 영화 장르: 585    [Action, Sci-Fi, Thriller]
Name: genre, dtype: object
영화이름: Galaxy Quest (1999), 영화 장르: 3106    [Adventure, Comedy, Sci-Fi]
Name: genre, dtype: object
영화이름: Matrix, The (1999), 영화 장르: 2502    [Action, Sci-Fi, Thriller]
Name: genre, dtype: object
영화이름: 

장르를 확인 했을때 좋아하는 영화와 비슷한 장르의 영화를 추천해주는 것 같습니다.

Forrest Gump이 추천에 기여한 정도를 확인

In [198]:
forrest = movie_to_idx['Forrest Gump (1994)']
explain = als_model.explain(user, csr_data, itemid=forrest)

In [199]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Star Wars: Episode VI - Return of the Jedi (1983)', 0.14754972838790423),
 ('Toy Story (1995)', 0.10785328732025583),
 ('Back to the Future (1985)', 0.08112341738627127),
 ('Men in Black (1997)', 0.0685402464473454),
 ('E.T. the Extra-Terrestrial (1982)', 0.01163690667237555)]

### 성능 향성을 위한 다양한 시도

### 2회차

In [200]:
als_model = AlternatingLeastSquares(factors=177, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [201]:
favorite_movie = 'Men in Black (1997)'
list_similar = get_similar_movie(favorite_movie)
list_similar

['Men in Black (1997)',
 'Jurassic Park (1993)',
 'Total Recall (1990)',
 'Terminator 2: Judgment Day (1991)',
 'Fifth Element, The (1997)',
 'Independence Day (ID4) (1996)']

In [202]:
genre(list_similar)

영화이름: Men in Black (1997), 영화 장르: 1539    [Action, Adventure, Comedy, Sci-Fi]
Name: genre, dtype: object
영화이름: Jurassic Park (1993), 영화 장르: 476    [Action, Adventure, Sci-Fi]
Name: genre, dtype: object
영화이름: Total Recall (1990), 영화 장르: 2847    [Action, Adventure, Sci-Fi, Thriller]
Name: genre, dtype: object
영화이름: Terminator 2: Judgment Day (1991), 영화 장르: 585    [Action, Sci-Fi, Thriller]
Name: genre, dtype: object
영화이름: Fifth Element, The (1997), 영화 장르: 1491    [Action, Sci-Fi]
Name: genre, dtype: object
영화이름: Independence Day (ID4) (1996), 영화 장르: 770    [Action, Sci-Fi, War]
Name: genre, dtype: object


In [211]:
def recommended(user, als_model, csr_data, n):
    movie_recommended = als_model.recommend(user, csr_data, N=n, filter_already_liked_items=True)
    list_title = [idx_to_movie[i[0]] for i in movie_recommended]
    for i, j in zip(list_title, movie_recommended):
        print(f'영화이름: {i}')
        print(f'예측값: {j}')
    return list_title

In [212]:
user = user_to_idx[7000]
list_title = recommended(user, als_model, csr_data, 20)

영화이름: Star Wars: Episode V - The Empire Strikes Back (1980)
예측값: (117, 0.565932)
영화이름: Jurassic Park (1993)
예측값: (107, 0.46125954)
영화이름: Star Wars: Episode IV - A New Hope (1977)
예측값: (44, 0.44537205)
영화이름: Bug's Life, A (1998)
예측값: (4, 0.3527474)
영화이름: Toy Story 2 (1999)
예측값: (50, 0.34206808)
영화이름: Terminator 2: Judgment Day (1991)
예측값: (92, 0.33329856)
영화이름: Matrix, The (1999)
예측값: (124, 0.2830366)
영화이름: Star Wars: Episode I - The Phantom Menace (1999)
예측값: (60, 0.24060518)
영화이름: Babe (1995)
예측값: (322, 0.23505934)
영화이름: Galaxy Quest (1999)
예측값: (670, 0.21821707)
영화이름: Ghostbusters (1984)
예측값: (243, 0.21777919)
영화이름: Big (1988)
예측값: (19, 0.21195619)
영화이름: Aladdin (1992)
예측값: (33, 0.19976874)
영화이름: Rain Man (1988)
예측값: (41, 0.18765897)
영화이름: Groundhog Day (1993)
예측값: (110, 0.18130188)
영화이름: Total Recall (1990)
예측값: (62, 0.1771789)
영화이름: Forrest Gump (1994)
예측값: (160, 0.17567731)
영화이름: Twelve Monkeys (1995)
예측값: (317, 0.174162)
영화이름: Ferris Bueller's Day Off (1986)
예측값: (13, 0.17406274)

In [213]:
genre(list_title)

영화이름: Star Wars: Episode V - The Empire Strikes Back (1980), 영화 장르: 1178    [Action, Adventure, Drama, Sci-Fi, War]
Name: genre, dtype: object
영화이름: Jurassic Park (1993), 영화 장르: 476    [Action, Adventure, Sci-Fi]
Name: genre, dtype: object
영화이름: Star Wars: Episode IV - A New Hope (1977), 영화 장르: 257    [Action, Adventure, Fantasy, Sci-Fi]
Name: genre, dtype: object
영화이름: Bug's Life, A (1998), 영화 장르: 2286    [Animation, Children's, Comedy]
Name: genre, dtype: object
영화이름: Toy Story 2 (1999), 영화 장르: 3045    [Animation, Children's, Comedy]
Name: genre, dtype: object
영화이름: Terminator 2: Judgment Day (1991), 영화 장르: 585    [Action, Sci-Fi, Thriller]
Name: genre, dtype: object
영화이름: Matrix, The (1999), 영화 장르: 2502    [Action, Sci-Fi, Thriller]
Name: genre, dtype: object
영화이름: Star Wars: Episode I - The Phantom Menace (1999), 영화 장르: 2559    [Action, Adventure, Fantasy, Sci-Fi]
Name: genre, dtype: object
영화이름: Babe (1995), 영화 장르: 33    [Children's, Comedy, Drama]
Name: genre, dtype: object
영화이름:

In [207]:
forrest = movie_to_idx['Jurassic Park (1993)']
explain = als_model.explain(user, csr_data, itemid=forrest)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Men in Black (1997)', 0.3687325948834784),
 ('Star Wars: Episode VI - Return of the Jedi (1983)', 0.04378854927283063),
 ('Back to the Future (1985)', 0.03285452671311708),
 ('E.T. the Extra-Terrestrial (1982)', 0.007217928508565457),
 ('Toy Story (1995)', 0.0018393829259490363)]

### 3회차

In [214]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.001, use_gpu=False, iterations=15, dtype=np.float32)
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [215]:
favorite_movie = 'Men in Black (1997)'
list_similar = get_similar_movie(favorite_movie)
list_similar

['Men in Black (1997)',
 'Jurassic Park (1993)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Matrix, The (1999)',
 'Fifth Element, The (1997)']

In [216]:
genre(list_similar)

영화이름: Men in Black (1997), 영화 장르: 1539    [Action, Adventure, Comedy, Sci-Fi]
Name: genre, dtype: object
영화이름: Jurassic Park (1993), 영화 장르: 476    [Action, Adventure, Sci-Fi]
Name: genre, dtype: object
영화이름: Terminator 2: Judgment Day (1991), 영화 장르: 585    [Action, Sci-Fi, Thriller]
Name: genre, dtype: object
영화이름: Total Recall (1990), 영화 장르: 2847    [Action, Adventure, Sci-Fi, Thriller]
Name: genre, dtype: object
영화이름: Matrix, The (1999), 영화 장르: 2502    [Action, Sci-Fi, Thriller]
Name: genre, dtype: object
영화이름: Fifth Element, The (1997), 영화 장르: 1491    [Action, Sci-Fi]
Name: genre, dtype: object


In [217]:
def recommended(user, als_model, csr_data, n):
    movie_recommended = als_model.recommend(user, csr_data, N=n, filter_already_liked_items=True)
    list_title = [idx_to_movie[i[0]] for i in movie_recommended]
    for i, j in zip(list_title, movie_recommended):
        print(f'영화이름: {i}')
        print(f'예측값: {j}')
    return list_title

In [218]:
user = user_to_idx[7000]
list_title = recommended(user, als_model, csr_data, 20)

영화이름: Star Wars: Episode V - The Empire Strikes Back (1980)
예측값: (117, 0.6183088)
영화이름: Star Wars: Episode IV - A New Hope (1977)
예측값: (44, 0.5438141)
영화이름: Toy Story 2 (1999)
예측값: (50, 0.41784924)
영화이름: Jurassic Park (1993)
예측값: (107, 0.41604868)
영화이름: Star Wars: Episode I - The Phantom Menace (1999)
예측값: (60, 0.37449965)
영화이름: Terminator 2: Judgment Day (1991)
예측값: (92, 0.36986202)
영화이름: Matrix, The (1999)
예측값: (124, 0.35813347)
영화이름: Galaxy Quest (1999)
예측값: (670, 0.35271615)
영화이름: Bug's Life, A (1998)
예측값: (4, 0.34284902)
영화이름: Forrest Gump (1994)
예측값: (160, 0.32587016)
영화이름: Raiders of the Lost Ark (1981)
예측값: (120, 0.26031104)
영화이름: Ghostbusters (1984)
예측값: (243, 0.2560404)
영화이름: American Beauty (1999)
예측값: (99, 0.25411418)
영화이름: Aladdin (1992)
예측값: (33, 0.2475217)
영화이름: 2001: A Space Odyssey (1968)
예측값: (602, 0.24566467)
영화이름: Close Encounters of the Third Kind (1977)
예측값: (91, 0.24032399)
영화이름: Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)
예측값: (55

In [219]:
genre(list_title)

영화이름: Star Wars: Episode V - The Empire Strikes Back (1980), 영화 장르: 1178    [Action, Adventure, Drama, Sci-Fi, War]
Name: genre, dtype: object
영화이름: Star Wars: Episode IV - A New Hope (1977), 영화 장르: 257    [Action, Adventure, Fantasy, Sci-Fi]
Name: genre, dtype: object
영화이름: Toy Story 2 (1999), 영화 장르: 3045    [Animation, Children's, Comedy]
Name: genre, dtype: object
영화이름: Jurassic Park (1993), 영화 장르: 476    [Action, Adventure, Sci-Fi]
Name: genre, dtype: object
영화이름: Star Wars: Episode I - The Phantom Menace (1999), 영화 장르: 2559    [Action, Adventure, Fantasy, Sci-Fi]
Name: genre, dtype: object
영화이름: Terminator 2: Judgment Day (1991), 영화 장르: 585    [Action, Sci-Fi, Thriller]
Name: genre, dtype: object
영화이름: Matrix, The (1999), 영화 장르: 2502    [Action, Sci-Fi, Thriller]
Name: genre, dtype: object
영화이름: Galaxy Quest (1999), 영화 장르: 3106    [Adventure, Comedy, Sci-Fi]
Name: genre, dtype: object
영화이름: Bug's Life, A (1998), 영화 장르: 2286    [Animation, Children's, Comedy]
Name: genre, dtype: ob

In [220]:
forrest = movie_to_idx['Jurassic Park (1993)']
explain = als_model.explain(user, csr_data, itemid=forrest)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Men in Black (1997)', 0.36587528987257806),
 ('Back to the Future (1985)', 0.05441776912711779),
 ('Star Wars: Episode VI - Return of the Jedi (1983)', 0.009794487779293856),
 ('E.T. the Extra-Terrestrial (1982)', 0.0061340975017939985),
 ('Toy Story (1995)', -0.024428524328053675)]

### 4회차

In [221]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=50, dtype=np.float32)
als_model.fit(csr_data_transpose)

  0%|          | 0/50 [00:00<?, ?it/s]

In [222]:
favorite_movie = 'Men in Black (1997)'
list_similar = get_similar_movie(favorite_movie)
list_similar

['Men in Black (1997)',
 'Jurassic Park (1993)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Independence Day (ID4) (1996)',
 'Matrix, The (1999)']

In [223]:
genre(list_similar)

영화이름: Men in Black (1997), 영화 장르: 1539    [Action, Adventure, Comedy, Sci-Fi]
Name: genre, dtype: object
영화이름: Jurassic Park (1993), 영화 장르: 476    [Action, Adventure, Sci-Fi]
Name: genre, dtype: object
영화이름: Terminator 2: Judgment Day (1991), 영화 장르: 585    [Action, Sci-Fi, Thriller]
Name: genre, dtype: object
영화이름: Total Recall (1990), 영화 장르: 2847    [Action, Adventure, Sci-Fi, Thriller]
Name: genre, dtype: object
영화이름: Independence Day (ID4) (1996), 영화 장르: 770    [Action, Sci-Fi, War]
Name: genre, dtype: object
영화이름: Matrix, The (1999), 영화 장르: 2502    [Action, Sci-Fi, Thriller]
Name: genre, dtype: object


In [224]:
def recommended(user, als_model, csr_data, n):
    movie_recommended = als_model.recommend(user, csr_data, N=n, filter_already_liked_items=True)
    list_title = [idx_to_movie[i[0]] for i in movie_recommended]
    for i, j in zip(list_title, movie_recommended):
        print(f'영화이름: {i}')
        print(f'예측값: {j}')
    return list_title

In [225]:
user = user_to_idx[7000]
list_title = recommended(user, als_model, csr_data, 20)

영화이름: Star Wars: Episode V - The Empire Strikes Back (1980)
예측값: (117, 0.6185671)
영화이름: Star Wars: Episode IV - A New Hope (1977)
예측값: (44, 0.568957)
영화이름: Star Wars: Episode I - The Phantom Menace (1999)
예측값: (60, 0.43328184)
영화이름: Jurassic Park (1993)
예측값: (107, 0.40727115)
영화이름: Toy Story 2 (1999)
예측값: (50, 0.40490407)
영화이름: Forrest Gump (1994)
예측값: (160, 0.40119916)
영화이름: Terminator 2: Judgment Day (1991)
예측값: (92, 0.3420034)
영화이름: Matrix, The (1999)
예측값: (124, 0.3396659)
영화이름: Bug's Life, A (1998)
예측값: (4, 0.32725877)
영화이름: Galaxy Quest (1999)
예측값: (670, 0.31170374)
영화이름: Raiders of the Lost Ark (1981)
예측값: (120, 0.302934)
영화이름: American Beauty (1999)
예측값: (99, 0.28598726)
영화이름: Babe (1995)
예측값: (322, 0.2714895)
영화이름: Groundhog Day (1993)
예측값: (110, 0.26669472)
영화이름: Close Encounters of the Third Kind (1977)
예측값: (91, 0.25639996)
영화이름: Shakespeare in Love (1998)
예측값: (126, 0.23484068)
영화이름: Aladdin (1992)
예측값: (33, 0.22916847)
영화이름: Braveheart (1995)
예측값: (87, 0.22571616)
영화이름: Pr

In [226]:
genre(list_title)

영화이름: Star Wars: Episode V - The Empire Strikes Back (1980), 영화 장르: 1178    [Action, Adventure, Drama, Sci-Fi, War]
Name: genre, dtype: object
영화이름: Star Wars: Episode IV - A New Hope (1977), 영화 장르: 257    [Action, Adventure, Fantasy, Sci-Fi]
Name: genre, dtype: object
영화이름: Star Wars: Episode I - The Phantom Menace (1999), 영화 장르: 2559    [Action, Adventure, Fantasy, Sci-Fi]
Name: genre, dtype: object
영화이름: Jurassic Park (1993), 영화 장르: 476    [Action, Adventure, Sci-Fi]
Name: genre, dtype: object
영화이름: Toy Story 2 (1999), 영화 장르: 3045    [Animation, Children's, Comedy]
Name: genre, dtype: object
영화이름: Forrest Gump (1994), 영화 장르: 352    [Comedy, Romance, War]
Name: genre, dtype: object
영화이름: Terminator 2: Judgment Day (1991), 영화 장르: 585    [Action, Sci-Fi, Thriller]
Name: genre, dtype: object
영화이름: Matrix, The (1999), 영화 장르: 2502    [Action, Sci-Fi, Thriller]
Name: genre, dtype: object
영화이름: Bug's Life, A (1998), 영화 장르: 2286    [Animation, Children's, Comedy]
Name: genre, dtype: object
영

In [229]:
forrest = movie_to_idx['Jurassic Park (1993)']
explain = als_model.explain(user, csr_data, itemid=forrest)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Men in Black (1997)', 0.36884384016793503),
 ('Back to the Future (1985)', 0.06476473807962599),
 ('Star Wars: Episode VI - Return of the Jedi (1983)', 0.016205543308612652),
 ('E.T. the Extra-Terrestrial (1982)', -0.016220598492671448),
 ('Toy Story (1995)', -0.02914159445799184)]

### 5회차

In [230]:
als_model = AlternatingLeastSquares(factors=177, regularization=0.001, use_gpu=False, iterations=15, dtype=np.float32)
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [231]:
favorite_movie = 'Men in Black (1997)'
list_similar = get_similar_movie(favorite_movie)
list_similar

['Men in Black (1997)',
 'Jurassic Park (1993)',
 'Total Recall (1990)',
 'Terminator 2: Judgment Day (1991)',
 'Agnes Browne (1999)',
 'Fifth Element, The (1997)']

In [232]:
genre(list_similar)

영화이름: Men in Black (1997), 영화 장르: 1539    [Action, Adventure, Comedy, Sci-Fi]
Name: genre, dtype: object
영화이름: Jurassic Park (1993), 영화 장르: 476    [Action, Adventure, Sci-Fi]
Name: genre, dtype: object
영화이름: Total Recall (1990), 영화 장르: 2847    [Action, Adventure, Sci-Fi, Thriller]
Name: genre, dtype: object
영화이름: Terminator 2: Judgment Day (1991), 영화 장르: 585    [Action, Sci-Fi, Thriller]
Name: genre, dtype: object
영화이름: Agnes Browne (1999), 영화 장르: 3055    [Comedy, Drama]
Name: genre, dtype: object
영화이름: Fifth Element, The (1997), 영화 장르: 1491    [Action, Sci-Fi]
Name: genre, dtype: object


In [233]:
def recommended(user, als_model, csr_data, n):
    movie_recommended = als_model.recommend(user, csr_data, N=n, filter_already_liked_items=True)
    list_title = [idx_to_movie[i[0]] for i in movie_recommended]
    for i, j in zip(list_title, movie_recommended):
        print(f'영화이름: {i}')
        print(f'예측값: {j}')
    return list_title

In [234]:
user = user_to_idx[7000]
list_title = recommended(user, als_model, csr_data, 20)

영화이름: Star Wars: Episode V - The Empire Strikes Back (1980)
예측값: (117, 0.5844353)
영화이름: Jurassic Park (1993)
예측값: (107, 0.434484)
영화이름: Star Wars: Episode IV - A New Hope (1977)
예측값: (44, 0.4199424)
영화이름: Toy Story 2 (1999)
예측값: (50, 0.4063574)
영화이름: Terminator 2: Judgment Day (1991)
예측값: (92, 0.29381618)
영화이름: Star Wars: Episode I - The Phantom Menace (1999)
예측값: (60, 0.29019636)
영화이름: Galaxy Quest (1999)
예측값: (670, 0.26241738)
영화이름: Bug's Life, A (1998)
예측값: (4, 0.23293328)
영화이름: Matrix, The (1999)
예측값: (124, 0.2318412)
영화이름: Babe (1995)
예측값: (322, 0.2265811)
영화이름: Big (1988)
예측값: (19, 0.19704497)
영화이름: Silence of the Lambs, The (1991)
예측값: (121, 0.19325708)
영화이름: Princess Bride, The (1987)
예측값: (5, 0.18093623)
영화이름: Close Encounters of the Third Kind (1977)
예측값: (91, 0.1769485)
영화이름: Ferris Bueller's Day Off (1986)
예측값: (13, 0.1738518)
영화이름: Total Recall (1990)
예측값: (62, 0.17157684)
영화이름: Contact (1997)
예측값: (479, 0.16779624)
영화이름: Aladdin (1992)
예측값: (33, 0.16496031)
영화이름: Terminat

In [235]:
genre(list_title)

영화이름: Star Wars: Episode V - The Empire Strikes Back (1980), 영화 장르: 1178    [Action, Adventure, Drama, Sci-Fi, War]
Name: genre, dtype: object
영화이름: Jurassic Park (1993), 영화 장르: 476    [Action, Adventure, Sci-Fi]
Name: genre, dtype: object
영화이름: Star Wars: Episode IV - A New Hope (1977), 영화 장르: 257    [Action, Adventure, Fantasy, Sci-Fi]
Name: genre, dtype: object
영화이름: Toy Story 2 (1999), 영화 장르: 3045    [Animation, Children's, Comedy]
Name: genre, dtype: object
영화이름: Terminator 2: Judgment Day (1991), 영화 장르: 585    [Action, Sci-Fi, Thriller]
Name: genre, dtype: object
영화이름: Star Wars: Episode I - The Phantom Menace (1999), 영화 장르: 2559    [Action, Adventure, Fantasy, Sci-Fi]
Name: genre, dtype: object
영화이름: Galaxy Quest (1999), 영화 장르: 3106    [Adventure, Comedy, Sci-Fi]
Name: genre, dtype: object
영화이름: Bug's Life, A (1998), 영화 장르: 2286    [Animation, Children's, Comedy]
Name: genre, dtype: object
영화이름: Matrix, The (1999), 영화 장르: 2502    [Action, Sci-Fi, Thriller]
Name: genre, dtype: ob

In [229]:
forrest = movie_to_idx['Jurassic Park (1993)']
explain = als_model.explain(user, csr_data, itemid=forrest)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Men in Black (1997)', 0.36884384016793503),
 ('Back to the Future (1985)', 0.06476473807962599),
 ('Star Wars: Episode VI - Return of the Jedi (1983)', 0.016205543308612652),
 ('E.T. the Extra-Terrestrial (1982)', -0.016220598492671448),
 ('Toy Story (1995)', -0.02914159445799184)]

## 회고

||1회차|2회차|3회차|4회차|5회차|
|:------|:------|:------|:------|:------|:------|
|factors|100|177|100|100|177|
|regularization|0.01|0.01|0.001|0.01|0.001|
|iterations|15|15|15|50|15|

- 1회차 결과와 2회차 결과가 달라졌지만, 장르를 비교 했을 때는 크게 다르지는 않은 것 같습니다.
- 3회차 결과에서는 Jurassic Park에 영향을 준 결과를 보면 Toy Story가 - 값을 준 것을 알 수 있다.
    - Toy Story와 Jurassic Park가 장르에서 비슷한 점이 없어서 이러한 결과가 나온 것 같습니다.
    - 하지만 맨인블랙과 연관성이 높아 상위권에 있는 것 같습니다.
- 4회차 결과 성능이 향상되지 않은 것 같습니다.
- 5회차는 예측 숫자로 보는 경우 그 전 회차보다 낮아진 것을 볼 수 있다.